# Probability Exercise

<div align="right"><button><a href="https://colab.research.google.com/github/QuantEcon/africa-summer-course-2024/blob/main/exercises/day-08a/exercise_set_8_with_solution.ipynb"><img src="" heght="10px"/><img
  src="https://colab.research.google.com/assets/colab-badge.svg"
  alt="open with Colab" width="100px"/></a></button></div>

#### Written for the QuantEcon Africa Workshop (July 2024)
#### Author: [Smit Lunagariya](https://github.com/Smit-create),  [Shu Hu](https://shu-hu.com/)

This notebook provides some exercises on basic probability concepts.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import quantecon as qe

### Exercise 1

**Question 1.1**: Consider you have $n$ cards numbered from $1$ to $n$. You need to complete the following function that returns the probability of getting an odd-numbered card.

In [ ]:
def odd_card_probability(n):
    possible_odds = n//2
    if n%2 == 1:
        possible_odds += 1
    return possible_odds/n

In [ ]:
# Test the solution
try:
    assert abs(odd_card_probability(5) - 3/5) < 1e-12
    assert abs(odd_card_probability(240) - 1/2) < 1e-12
    print("Congratulations!")
except:
    print("Wrong answer, please check your code again.")

### Exercise 2

The **Newcomb–Benford law** fits  many data sets, e.g., reports of incomes to tax authorities, in which
the leading digit is more likely to be small than large.

See [Benford's law](https://en.wikipedia.org/wiki/Benford%27s_law)

A Benford probability distribution is

$$
\textrm{Prob}\{X=d\}=\log _{10}(d+1)-\log _{10}(d)=\log _{10}\left(1+\frac{1}{d}\right)
$$

where $ d\in\{1,2,\cdots,9\} $.

**Question 2.1**: Write a function that returns the probability at any given point $d$ using the Benford probability distribution.

In [ ]:
def probability_benford(d):
    return np.log10(1 + 1/d)

In [ ]:
# Test the solution
try:
    assert abs(probability_benford(2) - 0.17609125905568124) < 1e-12
    print("Congratulations!")
except:
    print("Wrong answer, please check your code again.")

**Question 2.2**: Using the above function, write a function that returns the sum of probabilities at all the points in the state space of Benford's distribution i.e $d \in \{1, 2, ... 9\}$. This function will help us to verify that the sum of probabilities sum to $1$.

$$
\quad\sum_{d=1}^{9}\textrm{Prob}\{X=d\}=1
$$

In [ ]:
def test_probability_benford():
    return sum(probability_benford(d) for d in range(1, 10))

In [ ]:
# Test the solution
try:
    assert abs(test_probability_benford() - 1.0) < 1e-12
    print("Congratulations!")
except:
    print("Wrong answer, please check your code again.")

**Question 2.3**: Using the above given probability distribution function, compute the cumulative density distribution. Also, plot the PMF and CDF in the same graph.

*Hint 1: If $ X $ ia a random variable then CDF $ F_X(x)=F(x)=\textrm{Prob}\{X\le x\} $.*

*Hint 2: See the documentation of [numpy.cumsum](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html)*

In [ ]:
fig, ax = plt.subplots()

benford_pmf = np.array([probability_benford(d) for d in range(1,10)])
benford_cdf = np.cumsum(benford_pmf)

ax.plot(range(1,10), benford_pmf, 'o', label="PMF")
ax.plot(range(1,10), benford_cdf, '*', label="CDF")
plt.title('Benford\'s distribution')
plt.legend()
plt.show()

### Exercise 3

Consider the following joint distribution over $(X, Y)$ as:

$$
F=[f_{ij}]=\left[\begin{array}{cc}
0.2 & 0.15 & 0.15\\
0.1 & 0.15  & 0.05\\
0.025 & 0.025 & 0.15
\end{array}\right]
$$

In [ ]:
F = np.array([
  [0.2, 0.15, 0.15],
  [0.1, 0.15, 0.05],
  [0.025, 0.025, 0.15]
])

**Question 3.1**: Write two functions that help to calculate the marginal distribution for $\textrm{Prob}(X=i)$ and $\textrm{Prob}(Y=j)$ respectively.

*Hint: See the documentation of [numpy.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html). Read about `axis` argument.*

In [ ]:
def marginal_x(i):
    return F.sum(axis=1)[i]

def marginal_y(j):
    return F.sum(axis=0)[j]

In [ ]:
# Test the solution
try:
    assert abs(marginal_x(0) - 0.5) < 1e-12
    assert abs(marginal_y(0) - 0.325) < 1e-12
    print("Congratulations!")
except:
    print("Wrong answer, please check your code again.")

**Question 3.2**: Using the above two functions, write two new functions for computing the conditional distribution of $\textrm{Prob}\{X=i\vert Y=j\}$ and $\textrm{Prob}\{Y=i\vert X=j\}$ respectively.




In [ ]:
def conditional_x_given_y(i, j): # Prob {X = i | Y = j}
    return F[i][j]/marginal_y(j)

def conditional_y_given_x(i, j): # Prob {Y = i | X = j}
    return F[j][i]/marginal_x(j)

In [ ]:
# Test the solution
try:
    assert abs(conditional_x_given_y(0, 1) - 0.4615384615384615) < 1e-12
    assert abs(conditional_y_given_x(2, 0) - 0.3) < 1e-12
    print("Congratulations!")
except:
    print("Wrong answer, please check your code again.")

### Exercise 4

Generate 100000 data points from the [exponential distribution](https://en.wikipedia.org/wiki/Exponential_distribution) with density

$$
f(x; \alpha) = \alpha \exp(-\alpha x)
\qquad
(x > 0, \alpha > 0)
$$

taking $\alpha = 0.5$. Then

1. Plot a histogram of your sample and compare it to the density of the exponential distribution.
2. After looking up the maximum likelihood estimator of $\alpha$, compute the estimate given your data and check that it is in fact close to $\alpha$.

### Solution

After checking [the docs for the exponential distribution](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.expon.html) we proceed as follows

In [ ]:
from scipy.stats import expon
import numpy as np

alpha = 0.5
n = int(1e5)
# Scale controls the exponential parameter
ep = expon(scale=1.0/alpha)
# Generate n randome variables
x = ep.rvs(size=n)

Here's a histogram and density.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
xmin, xmax = 0.001, 10.0
ax.set_xlim(xmin, xmax)
ax.hist(x, density=True, bins=60, alpha=0.3)
grid = np.linspace(xmin, xmax, 200)
ax.plot(grid, ep.pdf(grid), 'k-', lw=2, label='true density')
ax.legend()
plt.show()

It's [well-known](http://en.wikipedia.org/wiki/Exponential_distribution) that the MLE of $\alpha$ is $1/\bar x$ where $\bar x$ is the mean of the sample.  Let's check that it is indeed close to $\alpha$.

In [ ]:
alpha_mle = 1.0 / x.mean()
print(f"max likelihood estimate of alpha is {alpha_mle}.")

### Exercise 5 (Markov chains)

Read the 
- lecture [Markov Chains: Basic Concepts](https://intro.quantecon.org/markov_chains_I.html) and 
- the [corresponding methods in the QuantEcon package](https://quanteconpy.readthedocs.io/en/latest/markov/core.html) 

before you attempt the following exercises.

**Exercise 5.1**

Using a method from the [QuantEcon package](https://quanteconpy.readthedocs.io/en/latest/markov/core.html), define a Markov chain object, called ``mc``, characterised by a stochastic matrix ``P`` and state values ``state_values``

In [ ]:
P = [[0.1, 0.3, 0.2, 0.4],
     [0.1, 0.4, 0.1, 0.4], 
     [0.3, 0.2, 0.3, 0.2], 
     [0.2, 0.1, 0.2, 0.5]]

state_values = ["recession", "slump", "recovery", "boom"]

#### Solution


In [ ]:
T = 5
mc = qe.MarkovChain(P, state_values=state_values)

**Exercise 5.2**

Simulate a Markov chain with length of ``T=5`` using the Markov chain defined from Exercise 5.1 with an initial value ``initial_value``.

In [ ]:
initial_value = "recession"

#### Solution


In [ ]:
mc.simulate(ts_length=T, init=initial_value)

**Exercise 5.3**

Calculate the stationary distribution(s), called ``ψ_star``, for the Markov chain defined from Exercise 5.1.

#### Solution


In [ ]:
ψ_star = mc.stationary_distributions[0]
ψ_star

**Exercise 5.4**

With the Markov chain defined from Exercise 5.1., compute the marignal distribution $\psi_t = \psi_0 P^t$ with ``t=100`` and an initial distribution ``ψ_0``.

In [ ]:
ψ_0 = [0.1, 0.4, 0.3, 0.2]

Compare $\psi_t$ with the ``ψ_star`` from Exercise 5.3.

#### Solution


In [ ]:
t = 100
P_power = np.linalg.matrix_power(P, t)
ψ = ψ_0 @ P_power
ψ

Read
- lecture [Markov Chains: Irreducibility and Ergodicity](https://intro.quantecon.org/markov_chains_II.html)


before you attempt the following exercises.

**Exercise 5.5**

With the Markov chain defined from Exercise 5.1., check whether its stochastic matrix is irreducible.

#### Solution


In [ ]:
mc.is_irreducible

**Exercise 5.6**

With the Markov chain defined from Exercise 5.1.,
- simulate a path of length ``T = 100_000`` for it,
- calculate the fraction of time spent on each state values, called ``p_hats`` and
- compare it with the stationary distribution we computed from Exercise 5.3.


#### Solution


In [ ]:
T = 100_000
X = mc.simulate(ts_length=T)

In [ ]:
p_hats = []
for x0 in state_values:
    p_hat = (X == x0).cumsum() / (1 + np.arange(T, dtype=float))
    p_hats.append(p_hat[-1])

In [ ]:
p_hats

In [ ]:
ψ_star